# Importing Franke's function
Including a plot

# My notes:
- b and d: leave intercept out
- when to use sudo

In [ ]:
%reset -f

%matplotlib inline

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import random, seed
import pandas as pd

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Generate the data
x = np.arange(0, 1, 0.05)
y = np.arange(0, 1, 0.05)
x, y = np.meshgrid(x,y)


def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4


z = FrankeFunction(x, y)

# Plot the surface.
surf = ax.plot_surface(x, y, z, cmap=cm.coolwarm, linewidth=0, antialiased=False)

# Customize the z axis.
ax.set_zlim(-0.10, 1.40)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

# Exercise a)

In this exercise, we only applied limited scaling of the data by removing the intercept. Removing the intercept mainly aims to make the code more in line with the following codes for the Ridge and Lasso Regression, where removing the intercept has an impact on the cost function.
Since our produced data is evenly distributed in the intervall [0,1), there is no need to identify outliers.

We perform splitting the data in test and train data once, and then apply the OLS fitting with polynomials for each degree. The uniformly splitting of the data allows us a better comparison of the MSEs later on.

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

def rsquare(y, ypredict):
    return 1-np.sum((y-ypredict)**2)/np.sum((y-np.mean(y))**2)

def MSE(y, ypredict):
    return 1/len(y)*(y.T-ypredict.T) @ (y-ypredict)

# number of data points
n = 10
# degree of the fitted polynomial
degree = 2

# setting up the Designmatrix

x = np.linspace(0, 1, n)
y = np.linspace(0, 1, n)

def create_X(x,y,degree):
    X = np.zeros((n, int((degree+2)*(degree+1)/2)))
    for i in range(0, degree+1):# +1
        for k in range(0, i+1):#+1
            if i>0:
                indx = int(i*(i+1)/2)
            else: indx=0
            X[:,indx+k] = (x**k)*(y**(i-k))#+1
    return X

X = create_X(x,y,degree)

# scaling / centering of the data
intercept = np.mean(X[:,0])
X = X[:,1:len(X)]
y = y - intercept

# splitting in train and test

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
x = X_train[:,1].reshape((len(X_train)),1)

# OLS fitting without stochastic noise

beta = np.zeros((int((degree+2)*(degree+1)/2), int((degree+2)*(degree+1)/2)))
MSE_train = np.zeros(degree)
MSE_test = np.zeros(degree)
rsquare_train = np.zeros((degree-1, 1))
rsquare_test = np.zeros((degree-1, 1))


for i in range(1, degree+1):
        c = int((i+2)*(i+1)/2)
        X_tilde = X_train[:,0:c-1]
        beta[0:c-1, i-1] = np.linalg.pinv(X_tilde.T @ X_tilde) @ X_tilde.T @ y_train
        ypredict = X_tilde @ beta[0:c-1, i-1]
        help =  MSE(y_train, ypredict)

        ypredict_test = X_test[:,0:c-1] @ beta[0:c-1, i-1]
        y_train = y_train.reshape(len(y_train), 1)
        MSE_train[i-1] = MSE(y_train, ypredict)

        rsquare_ = rsquare(y_train, ypredict)
        y_test = y_test.reshape(len(y_test), 1)
        MSE_test[i-1] = MSE(y_test, ypredict_test)


# with stochastic noise

# with stochastic input/ with linspace -> Satz von Peano ;)

# MSE and R^2


# plot the MSE and R^2 - as well as beta - as functions of the polynomial degree

ValueError: setting an array element with a sequence.